# Topic Modeling

Christoph & Nadia

## LDA (Latent Dirichlet Allocation)

## Imports von hauptsächlich Gensim & SpaCy

In [2]:
from pprint import pprint
import gensim
from gensim import corpora, models, similarities
from gensim.models import LdaModel
from spacy_preprocessing.preprocess import Preprocess
from tabulate import tabulate

import numpy as np
import pickle
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore')

from ipywidgets import widgets
from IPython.display import display, clear_output

from spacy_preprocessing.preprocess import Preprocess
import datetime

## Zusätzliche Preprocessing Funktionen

In [3]:
def is_blacklisted(word):
    return word in [
        'polizei', 'polizist', 'beamter', 'nr.', 'berlin', 'uhr', 'polizeimeldung',
        'nicht', 'jahr', 'jährige', 'jährig', 'jähriger', 'polizeiliche', 'polizeilich', '2015', '2016',
        '2014', '2017', '2018', 'polizeibeamter', '-', 'u.a.', 'z.b.', 'der', 'die', 'das', 'dem', 'den', 'diese', 'dieser',
        'diesen', 'diesem', 'um', 'für', 'eine', 'ein', 'einer', 'einen', 'einem', 'anderer', 'andere', 'anderen', 'anders'
    ]

def is_empty(word):
    return word.strip() == ''

def can_parse_date(word):
    try:
        datetime.datetime.strptime(word, '%d.%m.%Y')
        return True
    except ValueError:
        return False

def can_parse_num_int(word):
    try:
        int(word)
        return True
    except ValueError:
        return False
    
def can_parse_num_float(word):
    try:
        float(word)
        return True
    except ValueError:
        return False
    
def preprocess_after(doc):
    return [
        word 
        for word in doc
        if not is_empty(word)
        and not is_blacklisted(word)
        and not can_parse_date(word)
        and not can_parse_num_int(word)
        and not can_parse_num_float(word)
    ]

## Daten aus pickle Datei laden und Preprocessing durchführen

In [4]:
items = pickle.load(open('./data/items.pkl', 'rb'))
texts = [report['text'] for report in items]
titles = [report['title'] for report in items] 
data = [report['text_pre_processed_v1'] for report in items]

In [5]:
clean_data = [preprocess_after(doc) for doc in data]

## Dictionary und Bag of Words aus jedem Dokument erstellen

In [7]:
id2word = corpora.Dictionary(clean_data)
corpus = [id2word.doc2bow(doc) for doc in clean_data]

## LDA Modell

In [8]:
tmust

In [9]:
pprint(lda_model.print_topics())

[(1,
  '0.021*"hund" + 0.017*"wache" + 0.013*"corsa" + 0.011*"mangel" + '
  '0.010*"verspüren" + 0.010*"besitzerin" + 0.010*"polizeipräsident" + '
  '0.009*"rathenower" + 0.009*"gesteuert" + 0.009*"wilhelmshavener"'),
 (23,
  '0.023*"straße" + 0.009*"finden" + 0.009*"folgend" + 0.008*"sicherheit" + '
  '0.007*"information" + 0.007*"informieren" + 0.007*"platz" + '
  '0.006*"insgesamt" + 0.006*"berliner" + 0.006*"thema"'),
 (24,
  '0.079*"spandau" + 0.025*"reinickendorf" + 0.018*"residenzstraße" + '
  '0.017*"wechselgeld" + 0.015*"tipp" + 0.014*"silvesterrakete" + '
  '0.014*"brunsbütteler" + 0.014*"damm" + 0.013*"bisherig" + 0.012*"bleiben"'),
 (16,
  '0.032*"person" + 0.015*"trinkgelage" + 0.014*"aufzug" + '
  '0.011*"umherfliegend" + 0.011*"teilnehmer" + 0.010*"insgesamt" + '
  '0.010*"einsatzkraft" + 0.009*"verletzen" + 0.009*"straße" + '
  '0.008*"einsatz"'),
 (12,
  '0.046*"staatsanwaltschaft" + 0.025*"gemeinsame" + 0.025*"meldung" + '
  '0.023*"ermittlung" + 0.019*"landeskriminal

## Topics visualisieren

In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
21     0.173314 -0.214801       1        1  15.581396
3      0.225206  0.044041       2        1  13.742111
15     0.113822  0.146145       3        1   9.076159
18     0.168172  0.038536       4        1   7.175147
2      0.035138  0.186570       5        1   5.507417
19     0.092112 -0.047902       6        1   5.016666
10     0.076332 -0.214556       7        1   4.860227
8     -0.005917  0.182580       8        1   3.856475
6      0.013450  0.097640       9        1   3.807174
20    -0.015048 -0.150165      10        1   3.615525
4      0.159732 -0.023172      11        1   3.575918
5      0.012808  0.057512      12        1   3.249434
11     0.125151 -0.086686      13        1   3.176590
9      0.092644  0.102368      14        1   2.811402
23    -0.214666 -0.005188      15        1   2.259933
17     0.088672  0.021847      16        1   2.175944
12    -0.127976  0.111876      17        1   1.871902
7     -0.116981  0.083107      18        1   1.325694
16    -0.128603  0.001522      19        1   1.300631
14    -0.107292 -0.038204      20        1   1.287119
22    -0.078010 -0.027778      21        1   1.188401
13    -0.105478 -0.117886      22        1   1.125862
0     -0.178417 -0.058059      23        1   1.056605
1     -0.173103 -0.025335      24        1   0.771914
24    -0.125061 -0.064013      25        1   0.584350, topic_info=      Category         Freq               Term        Total  loglift  logprob
term                                                                         
160    Default  2565.000000              mitte  2565.000000  30.0000  30.0000
9      Default  3872.000000        krankenhaus  3872.000000  29.0000  29.0000
159    Default  9102.000000               mann  9102.000000  28.0000  28.0000
70     Default  2261.000000          feuerwehr  2261.000000  27.0000  27.0000
222    Default  1784.000000            wohnung  1784.000000  26.0000  26.0000
51     Default  1983.000000              täter  1983.000000  25.0000  25.0000
251    Default  1595.000000             fahrer  1595.000000  24.0000  24.0000
209    Default  2516.000000          stationär  2516.000000  23.0000  23.0000
33     Default  1576.000000               geld  1576.000000  22.0000  22.0000
207    Default  3340.000000             schwer  3340.000000  21.0000  21.0000
136    Default  3329.000000               frau  3329.000000  20.0000  20.0000
7      Default  3073.000000           flüchten  3073.000000  19.0000  19.0000
81     Default  1992.000000  landeskriminalamt  1992.000000  18.0000  18.0000
68     Default  7510.000000         ermittlung  7510.000000  17.0000  17.0000
182    Default  3013.000000           erleiden  3013.000000  16.0000  16.0000
751    Default  1160.000000        angestellte  1160.000000  15.0000  15.0000
1204   Default   733.000000            spandau   733.000000  14.0000  14.0000
19     Default  2462.000000         verletzung  2462.000000  13.0000  13.0000
229    Default  2216.000000               auto  2216.000000  12.0000  12.0000
631    Default  2099.000000           fahrzeug  2099.000000  11.0000  11.0000
17     Default  2950.000000         unbekannte  2950.000000  10.0000  10.0000
108    Default   906.000000        jugendliche   906.000000   9.0000   9.0000
2      Default  3136.000000         behandlung  3136.000000   8.0000   8.0000
347    Default  1865.000000         mutmaßlich  1865.000000   7.0000   7.0000
351    Default  1476.000000             person  1476.000000   6.0000   6.0000
758    Default  1154.000000             räuber  1154.000000   5.0000   5.0000
304    Default  1682.000000     verkehrsunfall  1682.000000   4.0000   4.0000
923    Default   863.000000            löschen   863.000000   3.0000   3.0000
69     Default   855.000000              feuer   855.000000   2.0000   2.0000
305    Default  4286.000000          verletzen  4286.000000   1.0000   1.0000
...

## MatrixSimilarity erstellen

In [11]:
index = similarities.MatrixSimilarity(lda_model[corpus])

## Die Suche

In [12]:
html_widget = widgets.HTML(
    value = ''
)

text_widget = widgets.Text()


def search(sender):  
    preprocess = Preprocess(sender.value)
    search_term_preprocessed = preprocess_after(preprocess.preprocess(sentence_split=False, with_pos=False))
    search_term_bow = id2word.doc2bow(search_term_preprocessed)
    search_term_lda = lda_model[search_term_bow]
    sims = index[search_term_lda]
    similarity_list = list(zip(range(len(sims)), sims, texts, titles))
    similarity_list.sort(key=lambda x: x[1], reverse=True)
    
    results = [f'<h3>{result[3]}</h3><em>Relevanz: {round(result[1] * 100, 2)} %</em><br><p style="width: 80%;">{result[2]}</p>' for result in similarity_list[:10]]
    output = '<hr><br>'.join(results)
    
    html_widget.value = output

text_widget.on_submit(search)    
    
display(text_widget)
display(html_widget)    


Text(value='')

HTML(value='')